In [1]:
# set the matplotlib backend so figures can be saved in the background
import os
import cv2
import pickle
import argparse
import numpy as np
import matplotlib.pyplot as plt
from imutils import paths
from keras.utils import np_utils
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from model.livenessnet import LivenessNet
import matplotlib

matplotlib.use("Agg")


# grab the list of images in our dataset directory, then initialize
# the list of data (i.e., images) and class images
imagePaths = list(paths.list_images("/home/harsh/my_liveness_detector/archive/real_vs_fake/real-vs-fake/train/"))
                  
print("[INFO] loading images...")

INIT_LR = 1e-4
BS = 10
EPOCHS = 50              
                  
data = []
labels = []



[INFO] loading images...


In [2]:
for imagePath in imagePaths:
    try:
        # extract the class label from the filename, load the image and
        # resize it to be a fixed 32x32 pixels, ignoring aspect ratio
        label = imagePath.split(os.path.sep)[-2]
        image = cv2.imread(imagePath)
        image = cv2.resize(image, (32, 32))

        # update the data and labels lists, respectively
        data.append(image)
        labels.append(label)

    except Exception as error:
        print("Image {} error".format(imagePath))




In [3]:
# convert the data into a NumPy array, then preprocess it by scaling
# all pixel intensities to the range [0, 1]
data = np.array(data, dtype="float") / 255.0

# encode the labels (which are currently strings) as integers and then
# one-hot encode them
# print(labels)
le = LabelEncoder()
labels = le.fit_transform(labels)
labels = np_utils.to_categorical(labels, 2)

# partition the data into training and testing splits using 75% of
# the data for training and the remaining 25% for testing
(trainX, testX, trainY, testY) = train_test_split(
    data, labels, test_size=0.25, random_state=42
)

# construct the training image generator for data augmentation
aug = ImageDataGenerator(
    rotation_range=20,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode="nearest",
)



In [4]:
# initialize the optimizer and model
print("[INFO] compiling model...")
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model = LivenessNet.build(width=32, height=32, depth=3,
                          classes=len(le.classes_))
model.compile(loss="binary_crossentropy", optimizer=opt, metrics=["accuracy"])

# train the network
print("[INFO] training network for {} epochs...".format(EPOCHS))
H = model.fit_generator(
    aug.flow(trainX, trainY, batch_size=BS),
    validation_data=(testX, testY),
    steps_per_epoch=len(trainX) // BS,
    epochs=EPOCHS,
)

# evaluate the network
print("[INFO] evaluating network...")
predictions = model.predict(testX, batch_size=BS)
print(
    classification_report(
        testY.argmax(axis=1), predictions.argmax(axis=1), target_names=le.classes_
    )
)

# save the network to disk
print("[INFO] serializing network to '{}'...".format('/home/harsh/my_liveness_detector/face_detector'))
model.save('/home/harsh/my_liveness_detector/face_detector')

# save the label encoder to disk
f = open('/home/harsh/my_liveness_detector/le.pickle', "wb")
f.write(pickle.dumps(le))
f.close()



[INFO] compiling model...


/home/harsh/anaconda3/lib/python3.7/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


[INFO] training network for 50 epochs...


/home/harsh/anaconda3/lib/python3.7/site-packages/keras/engine/training.py:1915: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/50
3598/3598 [==============================] - 414s 91ms/step - loss: 0.9039 - accuracy: 0.5155 - val_loss: 0.6722 - val_accuracy: 0.5885
Epoch 2/50
3598/3598 [==============================] - 286s 79ms/step - loss: 0.7270 - accuracy: 0.5538 - val_loss: 0.6506 - val_accuracy: 0.6239
Epoch 3/50
3598/3598 [==============================] - 289s 80ms/step - loss: 0.6846 - accuracy: 0.5804 - val_loss: 0.6235 - val_accuracy: 0.6662
Epoch 4/50
3598/3598 [==============================] - 291s 81ms/step - loss: 0.6647 - accuracy: 0.6051 - val_loss: 0.6122 - val_accuracy: 0.6695
Epoch 5/50
3598/3598 [==============================] - 289s 80ms/step - loss: 0.6449 - accuracy: 0.6290 - val_loss: 0.6128 - val_accuracy: 0.6631
Epoch 6/50
3598/3598 [==============================] - 291s 81ms/step - loss: 0.6386 - accuracy: 0.6362 - val_loss: 0.5933 - val_accuracy: 0.6935
Epoch 7/50
3598/3598 [==============================] - 290s 81ms/step - loss: 0.6273 - accuracy: 0.6521 - val_loss: 0

In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 16)        448       
_________________________________________________________________
activation (Activation)      (None, 32, 32, 16)        0         
_________________________________________________________________
batch_normalization (BatchNo (None, 32, 32, 16)        64        
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 16)        2320      
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 16)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 16)        64        
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 16)        0

In [7]:
# plot the training loss and accuracy
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, EPOCHS), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, EPOCHS), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, EPOCHS), H.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, EPOCHS), H.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy on Dataset")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.show()
plt.savefig('/home/harsh/my_liveness_detector/plot.jpg')


/home/harsh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  if sys.path[0] == '':
